In [1]:
import igraph
import pandas as pd
import os
import igraph as ig

In [2]:
%%time
edges_df = pd.read_pickle("/tmp/fc_engagement_fid_df.pkl")

CPU times: user 3.35 ms, sys: 2.76 s, total: 2.76 s
Wall time: 5.85 s


In [3]:
edges_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89659332 entries, 0 to 89659331
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   i       int64
 1   j       int64
 2   v       int64
dtypes: int64(3)
memory usage: 2.0 GB


In [4]:
edges_df.sample(5)

,i,j,v
40252767,244660,239,1
48499482,290132,293303,1
5338833,302101,426701,8
53832230,313796,1321,1
72458943,434736,10677,1


In [5]:
edges_df[edges_df['i']==2788].sample(5)

,i,j,v
20027965,2788,7009,7
22963636,2788,2248,8
25331632,2788,8558,1
6924279,2788,761,1
26684197,2788,742,1


In [6]:
%%time
g_pkl_filepath = "/tmp/fc_engagement_fid_ig.pkl"

if os.path.isfile(g_pkl_filepath):
    print("loading existing graph model")
    g = ig.Graph.Read_Pickle(gml_filepath)
else:
    g = ig.Graph.DataFrame(edges_df, directed=True, use_vids=False)

CPU times: user 2min 24s, sys: 1min 21s, total: 3min 46s
Wall time: 7min 26s


In [7]:
ig.summary(g)

IGRAPH DN-- 372097 89659332 -- 
+ attr: name (v), v (e)


In [8]:
def is_vertex(ig: igraph.GraphBase, fid:int) -> bool:
  try:
      ig.vs.find(name=fid)
      return True
  except:
      return False

In [9]:
def find_vertex(ig: igraph.GraphBase, fid:int) -> int:
    try:
      return ig.vs.find(name=fid)
    except:
      return None

In [10]:
def find_vertex_idx(ig: igraph.GraphBase, fid:int) -> int:
    try:
      return ig.vs.find(name=fid).index
    except:
      return None

In [11]:
vert = find_vertex(g, 2788)

In [18]:
find_vertex_idx(g, 2788)

2418

In [15]:
fids = [400000, 345324, 2788]
vids = [vid for fid in fids for vid in [find_vertex_idx(g, fid)] if vid is not None]

In [16]:
if len(vids) <= 0:
    print("invalid fid")

In [17]:
fids, vids

([400000, 345324, 2788], [341603, 23482, 2418])

In [23]:
fids = [2788]
vids = [vid for fid in fids for vid in [find_vertex_idx(g, fid)] if vid is not None]
fids, vids

([2788], [2418])

In [43]:
%%time
mindist = 1 # exclude the input vertex
order = 1
k1 = g.neighborhood(vids, order=order, mode="out", mindist=1)

CPU times: user 1.67 ms, sys: 502 µs, total: 2.17 ms
Wall time: 2.2 ms


In [39]:
len(k1), len(k1[0])

(1, 1239)

In [48]:
find_vertex_idx(g, 401218) in k1[0]

False

In [56]:
find_vertex_idx(g, 396659) in k1[0]

True

In [64]:
edges_df[(edges_df['i']==2788) & (edges_df['j'].isin([436542,396659, 345734, 390332, 507877, 515398, 516102, 379811]))]

,i,j,v
6486398,2788,396659,79


In [65]:
out_df = edges_df[edges_df['i']==2788].sort_values(by=['v'], ascending=False, inplace=False)

In [71]:
out_df['rank'] = out_df['v'].rank(ascending=False)

In [73]:
out_df[out_df['j']==396659]

,i,j,v,rank
6486398,2788,396659,79,91.0


In [75]:
len(edges_df[edges_df['i']==396659])

15373

In [76]:
len(edges_df[edges_df['j']==396659])

17561

In [77]:
%%time
mindist = 2 
order = 2
k2 = g.neighborhood(vids, order=order, mode="out", mindist=1)

CPU times: user 99.6 ms, sys: 616 ms, total: 716 ms
Wall time: 1.12 s


In [45]:
len(k2), len(k2[0])

(1, 103806)

In [50]:
find_vertex_idx(g, 401218) in k2[0]

True

In [51]:
find_vertex_idx(g, 436542) in k2[0]

True

In [52]:
find_vertex_idx(g, 292982) in k2[0]

True

In [13]:
%%time
mindist = 1 # exclude the input vertex
order = 1
k1 = g.neighborhood([3], order=order, mode="out", mindist=mindist)

CPU times: user 282 µs, sys: 16 µs, total: 298 µs
Wall time: 312 µs


In [14]:
len(k1[0])

990

In [15]:
%%time
mindist = 2 # exclude k1
order = 2
k2 = g.neighborhood([3], order=order, mode="out", mindist=mindist)

CPU times: user 3.4 ms, sys: 964 µs, total: 4.36 ms
Wall time: 5.68 ms


In [16]:
len(k2[0])

25150

In [17]:
%%time
mindist = 3 # exclude k2
order = 3
k3 = g.neighborhood([3], order=order, mode="out", mindist=mindist)

CPU times: user 13.9 ms, sys: 832 µs, total: 14.7 ms
Wall time: 19.7 ms


In [18]:
len(k3[0])

8363

In [19]:
%%time
mindist = 4 # exclude k3
order = 4
k4 = g.neighborhood([3], order=order, mode="out", mindist=mindist)

CPU times: user 12.9 ms, sys: 359 µs, total: 13.2 ms
Wall time: 13.5 ms


In [20]:
len(k4[0])

342

In [22]:
g.vs.find(name=3).index

4

In [21]:
g.vs.find(name=60).index

48

In [23]:
g.vs.find(name=5650).out_edges()[:5]

[igraph.Edge(<igraph.Graph object at 0x113ab2640>, 387832, {'v': 31}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 387833, {'v': 124}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 387834, {'v': 1}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 387835, {'v': 1}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 387836, {'v': 8})]

In [24]:
[edge for j in k1[0][:5] for edge in g.vs[j].out_edges() ]

[igraph.Edge(<igraph.Graph object at 0x113ab2640>, 0, {'v': 1}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102120, {'v': 13}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102121, {'v': 151}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102122, {'v': 706}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102123, {'v': 1}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102124, {'v': 1}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102125, {'v': 1}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102127, {'v': 3}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102128, {'v': 1}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102129, {'v': 5}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102131, {'v': 151}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102132, {'v': 1}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102133, {'v': 1}),
 igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102134, {'v': 1}),
 igraph.Edge(<igra

In [25]:
g.es[102121]

igraph.Edge(<igraph.Graph object at 0x113ab2640>, 102121, {'v': 151})

In [26]:
def into_ijv_for_edge(graph:ig.Graph, edge:ig.Edge, degree:int):
    return {
        'i': graph.vs[edge.tuple[0]]["name"],
        'j': graph.vs[edge.tuple[1]]["name"],
        'v': edge["v"],
        'k': degree
    }

In [27]:
def get_localtrust_with_degree(graph:ig.Graph, in_addr:str, limit_rows:int, max_degree:int) -> list: 
    lt = []
    mindist_and_order = 1
    limit = limit_rows
    while mindist_and_order <= max_degree:
        neighbors = graph.neighborhood(
            [in_addr], order=mindist_and_order, mode="out", mindist=mindist_and_order
        )
        k_edges = [edge for j in neighbors[0][:limit] for edge in graph.vs[j].out_edges()]
        
        [lt.append(into_ijv_for_edge(graph, e, mindist_and_order)) for e in k_edges]

        limit = limit - len(neighbors[0])
        if limit <= 0:
            break # we have reached limit of neighbors
        mindist_and_order += 1
    # end of while
    return lt

In [28]:
%%time
localtrust = get_localtrust_with_degree(graph=g,
                                        in_addr=3,
                                        limit_rows=10,
                                        max_degree=3)

CPU times: user 96.5 ms, sys: 19.4 ms, total: 116 ms
Wall time: 119 ms


In [29]:
len(localtrust)

25503

In [30]:
import random
random.sample(localtrust,5)

[{'i': 533, 'j': 2251, 'v': 9, 'k': 1},
 {'i': 4, 'j': 206, 'v': 1, 'k': 1},
 {'i': 533, 'j': 8615, 'v': 1, 'k': 1},
 {'i': 533, 'j': 191167, 'v': 1, 'k': 1},
 {'i': 533, 'j': 20204, 'v': 1, 'k': 1}]

In [31]:
%%time
localtrust = get_localtrust_with_degree(graph=g,
                                        in_addr=3,
                                        limit_rows=100,
                                        max_degree=5)

CPU times: user 158 ms, sys: 17 ms, total: 175 ms
Wall time: 264 ms


In [32]:
len(localtrust)

46127

In [33]:
import random
random.sample(localtrust, 5)

[{'i': 533, 'j': 978, 'v': 1, 'k': 1},
 {'i': 3, 'j': 12312, 'v': 38, 'k': 1},
 {'i': 533, 'j': 194470, 'v': 1, 'k': 1},
 {'i': 533, 'j': 11002, 'v': 1, 'k': 1},
 {'i': 533, 'j': 194287, 'v': 1, 'k': 1}]

In [34]:
%%time
localtrust = get_localtrust_with_degree(graph=g,
                                        in_addr=17866,
                                        limit_rows=100,
                                        max_degree=5)

CPU times: user 249 ms, sys: 15.3 ms, total: 265 ms
Wall time: 277 ms


In [35]:
len(localtrust)

69685

In [36]:
len(g.vs)

44787

In [37]:
g.vs[0]["name"]

533

In [38]:
%%time
from tqdm import tqdm
for v in tqdm(g.vs):
    get_localtrust_with_degree(
        graph=g,
        in_addr=v["name"],
        limit_rows=100,
        max_degree=5
    )

  0%|                                                                                                                                         | 27/44787 [00:06<2:46:22,  4.48it/s]


KeyboardInterrupt: 

In [61]:
def get_neighbors(graph:ig.Graph, in_addr:str, max_neighbors:int, max_degree:int) -> list:
    import itertools
    klists = []
    mindist_and_order = 1
    limit = max_neighbors
    while mindist_and_order <= max_degree:
        neighbors = graph.neighborhood(
            [in_addr], order=mindist_and_order, mode="out", mindist=mindist_and_order
        )
        klists.append(g.vs[neighbors[0][:limit]]["name"])
        
        limit = limit - len(neighbors[0])
        if limit <= 0:
            break # we have reached limit of neighbors
        mindist_and_order += 1
    # end of while
    return list(itertools.chain(*klists))

In [58]:
%%time
neighbors = get_neighbors(
                graph=g,
                in_addr=5650,
                max_neighbors=10000,
                max_degree=5
            )

CPU times: user 8.81 ms, sys: 1.35 ms, total: 10.2 ms
Wall time: 13.6 ms


In [59]:
sum(1 for _ in neighbors)

10000

In [60]:
random.sample(neighbors,10)

[1087, 5840, 194343, 8394, 1581, 8787, 1031, 18531, 11988, 12875]

In [51]:
neighbors

[2,
 3,
 8,
 12,
 18,
 25,
 37,
 56,
 80,
 129,
 194,
 206,
 207,
 347,
 369,
 378,
 457,
 516,
 534,
 576,
 617,
 651,
 742,
 1097,
 1237,
 1325,
 1356,
 1626,
 1766,
 1970,
 2074,
 2433,
 2689,
 3621,
 4179,
 4286,
 4378,
 4407,
 4409,
 4606,
 5650,
 5774,
 7048,
 8413,
 10259,
 11188,
 12370,
 16148,
 18989,
 9816,
 533,
 1,
 4,
 5,
 6,
 9,
 10,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 22,
 23,
 24,
 27,
 28,
 30,
 31,
 35,
 36,
 38,
 39,
 40,
 42,
 43,
 44,
 45,
 48,
 50,
 51,
 52,
 53,
 54,
 57,
 59,
 60,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 71,
 72,
 73,
 74,
 75,
 79,
 81,
 83,
 87,
 88,
 90,
 93,
 97,
 99,
 101,
 102,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 119,
 120,
 121,
 124,
 125,
 127,
 128,
 130,
 132,
 133,
 134,
 135,
 136,
 137,
 141,
 142,
 143,
 144,
 145,
 147,
 148,
 152,
 157,
 158,
 159,
 160,
 162,
 164,
 165,
 166,
 167,
 169,
 173,
 176,
 177,
 180,
 181,
 183,
 188,
 190,
 191,
 193,
 195,
 197,
 199,
 203,
 204,
 208,
 210,
 216,
 217,
 2

In [65]:
%%time
from tqdm import tqdm
for v in tqdm(g.vs):
    get_neighbors(
        graph=g,
        in_addr=v.index,
        max_neighbors=100,
        max_degree=5
    )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44787/44787 [03:19<00:00, 224.31it/s]


CPU times: user 38.3 s, sys: 2.99 s, total: 41.3 s
Wall time: 3min 19s


In [44]:
def get_neighbor_edges(graph:ig.Graph, in_addr:str, max_neighbors:int, max_degree:int) -> list:
    import itertools
    klists = []
    mindist_and_order = 1
    limit = max_neighbors
    while mindist_and_order <= max_degree:
        neighbors = graph.neighborhood(
            [in_addr], order=mindist_and_order, mode="out", mindist=mindist_and_order
        )
        klists.append([(edge.tuple[1], edge["v"]) for j in neighbors[0][:limit] for edge in graph.vs[j].out_edges()])
        
        limit = limit - len(neighbors[0])
        if limit <= 0:
            break # we have reached limit of neighbors
        mindist_and_order += 1
    # end of while
    return list(itertools.chain(*klists))

In [45]:
%%time
neighbors = get_neighbor_edges(
                graph=g,
                in_addr=17866,
                max_neighbors=1000,
                max_degree=5
            )

CPU times: user 206 ms, sys: 19.7 ms, total: 226 ms
Wall time: 236 ms


In [46]:
sum(1 for _ in neighbors)

337237

In [47]:
random.sample(neighbors, 5)

[(5018, 1), (1191, 3), (7396, 1), (4957, 1), (2180, 11)]

In [48]:
%%time
from tqdm import tqdm
for v in tqdm(g.vs):
    get_neighbor_edges(
        graph=g,
        in_addr=v["name"],
        max_neighbors=100,
        max_degree=5
    )

  0%|▌                                                                                                                                         | 189/44787 [00:08<32:32, 22.84it/s]


KeyboardInterrupt: 